## Notbook D: Adding motion to the simulation

#### Prerequisites:
- segmentation and motion fields
- acquisition template rawdata

#### Goals:
- Simulating a steady-state sequence including motion

#### Content overview: 
- generating a surrogate signal
- combining surrogate signal and motion model
- adding both cardiac and respiratory motion to the simulation
- storing the ground truth motion information

In [ ]:
# first we need to set up the simulation again

from pathlib import Path
import os
import numpy as np 

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15,7.5]

import sirf.Gadgetron as pMR
import sirf.Reg as pReg
import sirf.DynamicSimulation as pDS

import auxiliary_functions as aux


root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"
fpath_input = root_path / "Input"

fig_path = root_path / "Figures"
fig_path.mkdir(exist_ok=True)


In [ ]:
#these steps just repeat the previous notebook
# set up simulation as we know it from before
fname_xml = fpath_input / "XCAT_TissueParameters_XML.xml"
fname_segmentation = fpath_input / "segmentation.nii"

segmentation = pReg.NiftiImageData3D(str(fname_segmentation))
simulation = pDS.MRDynamicSimulation(segmentation, str(fname_xml))

fname_acquisition_template = fpath_input / "acquisition_template.h5"
acquisition_template = pMR.AcquisitionData(str(fname_acquisition_template))

num_acq_subset = acquisition_template.number()
subset = np.arange(num_acq_subset)
acquisition_template = acquisition_template.get_subset(subset)

# to activate a golden-angle 2D encoding model we only need to set the trajectory
acquisition_template = pMR.set_goldenangle2D_trajectory(acquisition_template)



In [ ]:
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_c_simulate_motion_static.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)

simulation.write_simulation_results(str(fname_output))


simulated_file = pMR.AcquisitionData(str(fname_output))
# recon_stat = aux.iterative_reconstruct_data(simulated_file)
recon_stat = aux.reconstruct_data(simulated_file)

recon_stat = pReg.NiftiImageData3D(recon_stat)
recon_stat.write('/media/sf_CCPPETMR/tmp_c_stat.nii')

### Motion Dynamics
To add motion to the simulation we need to construct an object called a MotionDynamic.
They consist of two things:
- a surrogate signal
- a set of displacement vector fields

Furthermore you need to define 
- how many motion states you want to simulate.
- where the ground truth should be stored to.

In [ ]:
Nt = 10000
t0_s = 0
tmax_s = 60* 10
f_Hz_resp = 0.25

t_resp, sig_resp = aux.get_normed_sinus_signal(t0_s, tmax_s, Nt, f_Hz_resp)

plt.figure()
plt.plot(t_resp[:400], sig_resp[:400])
plt.xlabel("time(sec)")
plt.ylabel("sig (a.u.)")
plt.title("Respiration signal")
plt.savefig(str(fig_path/"fig_c_respiratory_signal.png"))


In [ ]:

# configure the motion
num_motion_states = 5

# RESP
num_sim_resp_states = num_motion_states
resp_motion = pDS.MRMotionDynamic( num_sim_resp_states )
resp_motion.set_dynamic_signal(t_resp, sig_resp)
resp_motion.set_cyclicality(False)
resp_motion.set_groundtruth_folder_prefix(str(root_path / "Output/gt_resp/"))

aux.set_motionfields_from_path(resp_motion, str(fpath_input / 'mvfs_resp/'))
simulation.add_motion_dynamic(resp_motion)

In [ ]:
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_c_simulate_motion_breathing.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)

simulation.write_simulation_results(str(fname_output))

In [ ]:

simulated_file = pMR.AcquisitionData(str(fname_output))

# recon_resp = aux.iterative_reconstruct_data(simulated_file)
recon_resp = aux.reconstruct_data(simulated_file)

recon_resp = pReg.NiftiImageData3D(recon_resp)
recon_resp.write('/media/sf_CCPPETMR/tmp_c_resp.nii')

In [ ]:
Nt = 10000
t0_s = 0
tmax_s = 10*60
f_Hz_card = 1.25

t_card, sig_card = aux.get_normed_sawtooth_signal(t0_s, tmax_s, Nt, f_Hz_card)

plt.figure()
plt.plot(t_card[:400], sig_card[:400])
plt.xlabel("time(sec)")
plt.ylabel("sig (a.u.)")
plt.title("ECG")
plt.savefig(str(fig_path/"fig_c_cardiac_signal.png"))

__The order of motion dynamics matters!__

The more linear motion dynamic (i.e. the respiration) has to be added first. Otherwise the non-linear parts will influence each other.

In [ ]:
# CARD
num_sim_card_states = num_motion_states
card_motion = pDS.MRMotionDynamic( num_sim_card_states )
card_motion.set_dynamic_signal(t_card, sig_card)
card_motion.set_cyclicality(True)
card_motion.set_groundtruth_folder_prefix(str(root_path / "Output/gt_card/"))

aux.set_motionfields_from_path(card_motion, str(fpath_input / 'mvfs_card/'))
simulation.add_motion_dynamic(card_motion)



In [ ]:
simulation.simulate_data()

# now we simulate and 
fname_output = root_path / "Output/output_c_simulate_motion_cardio_respiratory.h5"
if not fname_output.parent.is_dir():
    fname_output.parent.mkdir(parents=True, exist_ok=True)

simulation.write_simulation_results(str(fname_output))

In [ ]:
print("Reading {}.".format(fname_output))
simulated_file = pMR.AcquisitionData(str(fname_output))

# recon_card = aux.iterative_reconstruct_data(simulated_file)
recon_card = aux.reconstruct_data(simulated_file)
recon_card = pReg.NiftiImageData3D(recon_card)
recon_card.write('/media/sf_CCPPETMR/tmp_c_cardresp.nii')

In [ ]:
f,ax = plt.subplots(1,3)
ax[0].imshow(np.transpose(np.abs(recon_stat.as_array())),cmap='gray')
ax[0].axis('off')
ax[0].set_title('No motion.')
ax[1].imshow(np.transpose(np.abs(recon_resp.as_array())),cmap='gray')
ax[1].axis('off')
ax[1].set_title('Respiratory motion.')
ax[2].imshow(np.transpose(np.abs(recon_card.as_array())),cmap='gray')
ax[2].axis('off')
ax[2].set_title('Cardio-respiratory motion.')

plt.savefig(str(fig_path/"fig_c_recon_motion_simulation.png"))

### Recap 
In this notebook we 
- introduced motion dynamics
- simulated respiratory motion
- simulated simultaneous cardiac and respiratory motion
- assessed the appearing artefacts in the reconstruction

__Up next: simulating a time-dependent magnetisation.__